In [1]:
from utils import transliterate, is_alpha, load_SET_dataset, load_twitter_dataset, get_N_tokens, LABELS, read_and_split_file
import pandas as pd
N = 5000

def load_tails():
    interim_dir = "/home/peterr/macocu/taskB/data/interim"
    texts, labels = list(), list()
    files = ["bswac_tail_pp", "cnrwac_tail_pp_corrected_2", "hrwac_tail_pp", "srwac_tail_pp"]
    langs = ["bs", "me", "hr", "sr"]

    for file, lang in zip(files, langs):
        full_path = os.path.join(interim_dir, file)
        current_texts = read_and_split_file(full_path)
        len_cur_texts = len(current_texts)
        texts.extend(current_texts)
        labels.extend([lang] * len_cur_texts)

    return pd.DataFrame(data={"text": texts, "labels": labels})

train = load_tails()
SET = load_SET_dataset().rename(columns={"language":"labels"})

dev_df = SET.loc[SET.split == "train", ["text", "labels"]]
test_df = SET.loc[SET.split != "train", ["text", "labels"]]



In [ ]:
def train_clf(N, train):
    from sklearn.svm import LinearSVC
    import gc
    from sklearn.feature_extraction.text import CountVectorizer
    gc.collect()
    clf = LinearSVC(dual=False)
    vectorizer = CountVectorizer(lowercase=True, binary=True, ngram_range=(3,6), analyzer="char", max_features=N)
    train_vectors = vectorizer.fit_transform(train.text)
    train_labels = train.labels
    clf.fit(train_vectors.toarray(), train_labels)

    return clf, vectorizer

def eval_clf(clf, eval_df, vectorizer):
    from sklearn.metrics import (
        f1_score,
        ConfusionMatrixDisplay,
        confusion_matrix,
        accuracy_score,
    )
    test_vectors = vectorizer.fit_transform(eval_df.text)
    y_true = eval_df.labels
    y_pred = clf.predict(test_vectors.toarray())
    macro = f1_score(y_true, y_pred, labels=LABELS, average="macro")
    micro = f1_score(y_true, y_pred, labels=LABELS, average="micro")
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)
    returndict = dict(
        N=N,
        macroF1=macro,
        microF1=micro,
        accuracy=acc,
        cm = cm,

    )
    return returndict
import numpy as np
results = list()
for N in [int(1e5)]:
    try:
        clf, vectorizer = train_clf(N, train)
        rezdict = eval_clf(clf, dev_df, vectorizer)
        rezdict["dev"] = "SET train"
        results.append(rezdict)
    except MemoryError:
        continue
pd.DataFrame(data=results).to_csv("2_part_SETIMES_hyperparams_optimization.csv")

In [7]:
pd.DataFrame(data=results)

,N,macroF1,microF1,accuracy,cm,dev
0,10,0.000000,0.000000,0.000000,"[[0, 0, 0, 2556], [0, 0, 0, 2556], [0, 0, 0, 2...",SET train
1,11,0.000000,0.000000,0.000000,"[[0, 0, 0, 2556], [0, 0, 0, 2556], [1, 0, 0, 2...",SET train
2,13,0.000000,0.000000,0.000000,"[[0, 0, 1, 2555], [1, 0, 0, 2555], [0, 0, 0, 2...",SET train
3,15,0.000221,0.000136,0.000136,"[[0, 0, 0, 2556], [1, 0, 0, 2555], [0, 0, 1, 2...",SET train
4,17,0.000221,0.000136,0.000136,"[[0, 0, 1, 2555], [2, 0, 0, 2554], [0, 0, 1, 2...",SET train
5,20,0.000221,0.000136,0.000136,"[[0, 0, 1, 2555], [2, 0, 0, 2554], [0, 0, 1, 2...",SET train
6,23,0.000221,0.000136,0.000136,"[[0, 0, 0, 2556], [3, 0, 0, 2553], [0, 0, 1, 2...",SET train
7,26,0.000221,0.000136,0.000136,"[[0, 0, 0, 2556], [3, 0, 0, 2553], [1, 0, 1, 2...",SET train
